# Arbeit mit Textdaten (Vertiefung Funktionen, Schleifen, RegEx)

## Ziel dieser Lern- und Übungseinheit

In diesem Notebook werden keine neuen Inhalte der Python-Programmiersprache eingeführt, sondern bereits gelernte Inhalte exemplarisch vertieft. Dafür werden am Beispiel der Parlamentsprotokolle des deutschen Bundestags Prinzipien der (einfachen) Textverarbeitung und -analyse in Python demonstriert. 

Derartige Textkorpora lassen sich mithilfe computerbasierter Methoden wie der des *Topic Modelling* bspw. hinsichtlich ihrer inhaltlich-thematischen Struktur untersuchen. In der geistes- und sozialwissenschaftlichen Forschung erfreuen sich Untersuchungen, die sich auf solche quantitative Verfahren stützen, in jüngster Zeit großer Beliebtheit (siehe u.a. Müller-Hansen et. al.). Die Plenarprotokolle der 20. Legislaturperiode sollen hier nun zunächst über die Bildung einfacher Häufigkeiten (engl. frequency) untersucht werden, denn:

> even though word frequency is about a simple quantitative phenomenon—the count of the occurrences of words in texts—it can produce fresh and interesting insights. Frequency analysis can help to both enrich our understanding of existing qualitative research problems and generate new questions. (McGillivray / Tóth, S. 36.)

Zu unterscheiden ist dabei zwischen der absoluten und relativen Worthäufigkeit sowie der relativen Dokumenthäufigkeit (vgl. ebd. S. 37 u. 44.):

- absolute Worthäufigkeit (im engl. auch raw frequency): einfache Zählung einzelner Worte
- relative Worthäufigkeit (im engl. relative oder normalized frequency): Worthäufigkeit geteilt durch die absolute Anzahl der Worte eines Texts 
- relative Dokumenthäufigkeit (im engl. relative document frequency oder auch relative DOCF): relative (prozentuale) Häufigkeit eines Wortes in einem Dokumentkorpus (ein Dokument könnte hier bspw. eine einzelne Rede sein)

Schwerpunktmäßig geht es anschließend um folgende Themen:

- Arbeit mit strukturierten Textformaten (hier in erster Linie JSON)
- Kapselung von Logik in Funktionen
- einfache Textmanipulation und -analyse 

## Referenzierte Literatur:

- McGillivray, Barbara, und Gábor Mihály Tóth. Applying Language Technology in Humanities Research: Design, Application, and the Underlying Logic. Springer International Publishing, 2020. DOI.org (Crossref), https://doi.org/10.1007/978-3-030-46493-6.
- Müller-Hansen, Finn, u. a. „Who Cares about Coal? Analyzing 70 Years of German Parliamentary Debates on Coal with Dynamic Topic Modeling“. Energy Research & Social Science, Bd. 72, Februar 2021, S. 101869. DOI.org (Crossref), https://doi.org/10.1016/j.erss.2020.101869.


**Führen Sie zunächst die nachfolgende Zelle aus, um die Beispieldaten herunterzuladen.**

In [1]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

sciebo_download = "https://uni-wuppertal.sciebo.de/s/OVSgl4lSy4AxBgt/download"
http_response = urlopen(sciebo_download)
with ZipFile(BytesIO(http_response.read())) as zip_file:
    zip_file.extractall(path='.')


In [1]:
# zunächst importieren wir das Modul json 
import json 

# dann öffnen wir beispielhaft eine Datei aus dem Ordner `pp20`
with open('pp20/20002.json', 'r', encoding='UTF-8') as file:
    # Verarbeitung der geöffneten JSON-Datei mit der Funktion `load`
    content = json.load(file)

# Ausgabe von content
content

{'date': '11.11.2021',
 'period': 'pp20',
 'number': 2,
 'speeches': [{'id': 'ID20200100',
   'speaker': {'forename': 'Marianne',
    'surname': 'Schieder',
    'party': 'SPD',
    'role': None},
   'text': 'Sehr geehrte Frau Präsidentin! Liebe Kolleginnen und Kollegen! Der Bundestag wird heute die Einsetzung eines Hauptausschusses beschließen. Das ist geübte Praxis. Bereits 2013 und 2017 hat der Bundestag direkt nach der Wahl einen solchen Hauptausschuss eingesetzt.\nDer Hauptausschuss soll den Zeitraum bis zur Konstituierung der ständigen Ausschüsse überbrücken. Er kann Vorlagen beraten, die ihm vom Plenum überwiesen werden, und er kann Beschlussempfehlungen für das Plenum erarbeiten. Damit sichert der Bundestag in einer Übergangsphase seine Handlungsfähigkeit. Es sollen diesem Ausschuss 31 Mitglieder angehören, und natürlich kommen dazu auch 31 stellvertretende Mitglieder. Die CDU/CSU-Fraktion möchte nun, dass nicht 31, sondern 39 Mitglieder in diesen Ausschuss berufen werden,\nDer 

In [2]:
first_speech = content['speeches'][0]['text']
first_speech

'Sehr geehrte Frau Präsidentin! Liebe Kolleginnen und Kollegen! Der Bundestag wird heute die Einsetzung eines Hauptausschusses beschließen. Das ist geübte Praxis. Bereits 2013 und 2017 hat der Bundestag direkt nach der Wahl einen solchen Hauptausschuss eingesetzt.\nDer Hauptausschuss soll den Zeitraum bis zur Konstituierung der ständigen Ausschüsse überbrücken. Er kann Vorlagen beraten, die ihm vom Plenum überwiesen werden, und er kann Beschlussempfehlungen für das Plenum erarbeiten. Damit sichert der Bundestag in einer Übergangsphase seine Handlungsfähigkeit. Es sollen diesem Ausschuss 31 Mitglieder angehören, und natürlich kommen dazu auch 31 stellvertretende Mitglieder. Die CDU/CSU-Fraktion möchte nun, dass nicht 31, sondern 39 Mitglieder in diesen Ausschuss berufen werden,\nDer Hauptausschuss wird voraussichtlich nur zweimal regulär tagen und zudem zwei Anhörungen durchführen. Er wird nur zwei Gesetze, zwei Verordnungen und eine Vorlage beraten, und dafür, meine ich, sollten doch 3

Dieser Inhalt ist nun der Variablen `first_speech` zugewiesen. Es handelt sich dabei um den eigentlichen Redetext ohne Hinweise des Bundestagspräsidiums oder Kommentare der anderen Abgeordneten. Im ursprünglichen Protokoll werden die Reden in Abschnitte unterteilt – die Markierung dieser Abschnitte ist auch hier noch erhalten (Zeichen: `\n`). Für die weitere Untersuchung sollen unerwünschte Zeichen wie bspw. `\n` aus dem Text entfernt und durch ein Leerzeichen ersetzt werden. Ergänzen Sie dazu in der nächsten Zelle die Funktion `remove_unwanted_chars`.

In [3]:
import re

def remove_unwanted_chars(text_input):
    """Die Methode `sub()` nimmt drei Parameter entgegen:
    1. Das Suchmuster 
    2. Womit das Muster ersetzt werden soll
    3. Den Textinhalt, auf dem diese Operation ausgeführt wird
    
    Es fehlt hier der dritte Parameter. Die Ersetzungen sollen auf text_input durchgeführt
    werden – ergo wird hier text_input eingesetzt.
    """
    
    text_cleaned = re.sub('[\n\t\r]', ' ', text_input)
    """Mithilfe des Keywords `return` wird der ‚errechnete‘ Wert zurückgegeben,
    in diesem Fall der Wert von text_cleaned"""
    return text_cleaned

In [4]:
try:
    cleaned_text = remove_unwanted_chars(first_speech)
    assert ' Herzlichen Dank' in cleaned_text
    print('Alles korrekt!')
except:
    print('Funktion lässt sich nicht aufrufen oder unerwünschte Zeichen werden nicht durch ein Leerzeichen ergänzt.')

Alles korrekt!


Gegeben sei zudem eine Funktion `tokenize_speech` (siehe hierzu die [04. Übungseinheit](04_modules-and-regex.ipynb)), welche den Redetext in einzelne Tokens (anhand der Leerzeichen unterteilt). Satzzeichen und Zahlen werden jedoch ignoriert.

In [5]:
def tokenize_speech(text_input):
    return re.findall("[A-Za-zÄäÜüÖöß]+", text_input)

Mit diesen beiden Hilfsfunktionen lässt sich nun der Redetext für die anschließende Untersuchung vorbereiten:

In [10]:
first_speech_cleaned_tokenized = tokenize_speech(remove_unwanted_chars(first_speech))
first_speech_cleaned_tokenized

['Sehr',
 'geehrte',
 'Frau',
 'Präsidentin',
 'Liebe',
 'Kolleginnen',
 'und',
 'Kollegen',
 'Der',
 'Bundestag',
 'wird',
 'heute',
 'die',
 'Einsetzung',
 'eines',
 'Hauptausschusses',
 'beschließen',
 'Das',
 'ist',
 'geübte',
 'Praxis',
 'Bereits',
 'und',
 'hat',
 'der',
 'Bundestag',
 'direkt',
 'nach',
 'der',
 'Wahl',
 'einen',
 'solchen',
 'Hauptausschuss',
 'eingesetzt',
 'Der',
 'Hauptausschuss',
 'soll',
 'den',
 'Zeitraum',
 'bis',
 'zur',
 'Konstituierung',
 'der',
 'ständigen',
 'Ausschüsse',
 'überbrücken',
 'Er',
 'kann',
 'Vorlagen',
 'beraten',
 'die',
 'ihm',
 'vom',
 'Plenum',
 'überwiesen',
 'werden',
 'und',
 'er',
 'kann',
 'Beschlussempfehlungen',
 'für',
 'das',
 'Plenum',
 'erarbeiten',
 'Damit',
 'sichert',
 'der',
 'Bundestag',
 'in',
 'einer',
 'Übergangsphase',
 'seine',
 'Handlungsfähigkeit',
 'Es',
 'sollen',
 'diesem',
 'Ausschuss',
 'Mitglieder',
 'angehören',
 'und',
 'natürlich',
 'kommen',
 'dazu',
 'auch',
 'stellvertretende',
 'Mitglieder',
 'Di

Auf Grundlage dieser Liste (`first_speech_cleaned_tokenized`) können wir nun erste Berechnungen durchführen. So lässt sich z.B. mithilfe der Funktion `len()` bestimmen, wie viele Worte (Tokens) diese erste Rede umfasst:

In [7]:
print(len(first_speech_cleaned_tokenized))

298


Um nun die absolute Häufigkeit eines jeden Worts (also eines Types) zu bestimmen, lässt sich die `Counter`-Klasse aus der Python-Standardbibliothek verwenden. Dieser wird mit der Initialisierung eine Liste übergeben – anschließend erfolgt die Zählung der Elemente in dieser Liste. Die Klasse beinhaltet zudem einige nützliche Methoden wie `most-common()` um die n-häufigsten Elemente auszugeben. Die Zählung und Ausgabe der fünf häufigsten Worte am Beispiel der ersten Rede sieht folgendermaßen aus:

In [11]:
# Import der Klasse `Counter` aus dem Modul `collections`
from collections import Counter

# Zählen der Tokens mit der Klasse `Counter`
tokens_counted = Counter(first_speech_cleaned_tokenized)

# Ausgabe der 5 häufigsten Tokens
tokens_counted.most_common(5)

[('der', 12),
 ('und', 10),
 ('Hauptausschuss', 9),
 ('Mitglieder', 7),
 ('die', 5)]

Auch ohne den genauen Inhalt der Rede der Abgeordneten Marianne Schieder zu kennen, liefert diese einfache Zählung einige Hinweise. So scheint es ganz offensichtlich um die Mitglieder für den [Hauptausschuss](https://www.bundestag.de/ausschuesse/hauptausschuss) zu gehen, der am Beginn der 20. Legislaturperiode vorübergehend eingerichtet wurde. Obwohl die anderen 3 Tokens (der, und, die) nicht sehr aussagekräftig sind, bietet diese Berechnung einen wertvollen Einblick. 

Nachfolgend soll für jede Rede die absolute sowie relative Häufigkeit berechnet werden. Ergänzen Sie dafür die Klasse `SpeechStats`, welche den in Tokens aufgesplitteten Text einer Rede als Argument entgegen nimmt und in den den beiden Attributen `count_abs` und `count_rel` die absolute sowie relative Häufigkeit als geordnete Liste von Token und entsprechender Häufigkeit (als Tuple) enthält.

In [12]:
"""Hinweis: Aufgabenstellung genau lesen! 
- im Attribut count_abs werden die absoluten Häufigkeiten je Token hinterlegt und zwar
als Liste bestehend Tuplen der Form (Token, Zähler)
- im Attribut count_rel hingegen soll die relative Häufigkeit hinterlegt werden und zwar
List mit Elementen der Form (Token, relativer Häufigkeitswert)
"""

class SpeechStats:

    def __init__(self, text_tokenized):
        tokens_counted = Counter(text_tokenized)
        # Hier muss auf die Länge von tokens_counted zugegriffen werden, damit wir
        # alle Inhalte abfragen können – alternativ könnten wir auch
        # `tokens_counted.items()` statt `tokens_counted.most_common()` verwenden
        self.count_abs = tokens_counted.most_common(len(tokens_counted))
        self.count_rel = []

        """
        Um die relative Häufigkeit pro Token ausrechnen zu können, muss hier
        über alle Tokens und ihre absolute Häufigkeit iteriert werden
        
        Der Ausdruck `for token, count in self.count_abs` weist die beiden
        Bestandteile der Tuple (Token, Zähler) den beiden Schleifenveriablen
        `token` und `count` zu, die wir dann innerhalb der Durchläufe der
        Schleife verwenden können.
        """
        for token, count in self.count_abs:
            """Die relative Häufigkeit ergibt sich durch absolute Häufigkeit geteilt durch
            Gesamtzahl aller Worte – die Gesamtzahl der Worte entspricht der Länge der
            Liste `text_tokenized`"""
            self.count_rel.append((token, count / len(text_tokenized)))

stats_first_speech = SpeechStats(first_speech_cleaned_tokenized)
stats_first_speech.count_rel[:10]

[('der', 0.040268456375838924),
 ('und', 0.03355704697986577),
 ('Hauptausschuss', 0.030201342281879196),
 ('Mitglieder', 0.02348993288590604),
 ('die', 0.016778523489932886),
 ('Der', 0.013422818791946308),
 ('wird', 0.013422818791946308),
 ('Ausschüsse', 0.013422818791946308),
 ('für', 0.013422818791946308),
 ('auch', 0.013422818791946308)]

In [13]:
try:
    assert isinstance(stats_first_speech, SpeechStats)
except:
    print('Die Klasse SpeechStats ist nicht korrekt definiert.')
try:
    assert len(stats_first_speech.count_rel) == 192 
except:
    print('Die Liste `count_rel` ist nicht korrekt definiert – sie sollte 192 Einträge haben.')
try:
    assert sum([i for _, i in stats_first_speech.count_rel[:10]]) < 0.5
    print("Alles korrekt!")
except:
    print('Die Summe der relativen Häufigkeiten der 10 häufigsten Tokens sollte kleiner als 0.5 sein.')

Alles korrekt!


Führen Sie diese Berechnungen nun für alle Reden der Datei `20002.json` (Variable `content`) durch. Extrahieren Sie dafür alle Reden aus der Variablen `content` und speichern die Reden samt der zugehörigen Statistiken in einem `dict` unter der jeweiligen ID. Das Dictionary soll folgende Struktur haben:

```python
{
    "id1": {
        "tokens": # Tokens der Rede
        "stats": # Objekt `SpeechStats` für die jeweilige Rede
    }, 
    "id2": {
        ...
    }
}
```

In [15]:
# Initialisierung eines leeren Dictionaries über den Ausdruck `{}`
speech_analysis = {}

# Iteration über alle Reden
for speech in content['speeches']:
    """
        Hier wird über alle ‚Reden‘ iteriert, die selbst ein `dict` sind. Zur Struktur
        siehe oben. Die einzelnen Informationen aus diesem `dict` lassen sich wiederum 
        durch Aufruf der jeweiligen Schlüssel extrahieren.
    """
    
    
    # Extraktion der ID der Rede, indem auf den Wert des Schlüssels `id` zugegriffen wird
    speech_id = speech['id']
    # Extraktion des Textes der Rede
    speech_text = speech['text']
    # Tokenisierung des Textes der Rede
    speech_text_tokenized = tokenize_speech(remove_unwanted_chars(speech_text))
    # Berechnung der statistischen Kennzahlen – als Parameter wird der bereinigte Text übergeben
    speech_stats = SpeechStats(speech_text_tokenized)
    # Speicherung von speech_text_tokenized und speech_stats in speech_analysis in einem eigenen Dictionary
    # dieses `dict` soll zwei Einträge haben, der Eintrag `tokens` fehlt und muss
    # noch angelegt werden
    speech_analysis[speech_id] = {'tokens': speech_text_tokenized, 'stats': speech_stats}

speech_analysis.items()

dict_items([('ID20200100', {'tokens': ['Sehr', 'geehrte', 'Frau', 'Präsidentin', 'Liebe', 'Kolleginnen', 'und', 'Kollegen', 'Der', 'Bundestag', 'wird', 'heute', 'die', 'Einsetzung', 'eines', 'Hauptausschusses', 'beschließen', 'Das', 'ist', 'geübte', 'Praxis', 'Bereits', 'und', 'hat', 'der', 'Bundestag', 'direkt', 'nach', 'der', 'Wahl', 'einen', 'solchen', 'Hauptausschuss', 'eingesetzt', 'Der', 'Hauptausschuss', 'soll', 'den', 'Zeitraum', 'bis', 'zur', 'Konstituierung', 'der', 'ständigen', 'Ausschüsse', 'überbrücken', 'Er', 'kann', 'Vorlagen', 'beraten', 'die', 'ihm', 'vom', 'Plenum', 'überwiesen', 'werden', 'und', 'er', 'kann', 'Beschlussempfehlungen', 'für', 'das', 'Plenum', 'erarbeiten', 'Damit', 'sichert', 'der', 'Bundestag', 'in', 'einer', 'Übergangsphase', 'seine', 'Handlungsfähigkeit', 'Es', 'sollen', 'diesem', 'Ausschuss', 'Mitglieder', 'angehören', 'und', 'natürlich', 'kommen', 'dazu', 'auch', 'stellvertretende', 'Mitglieder', 'Die', 'CDU', 'CSU', 'Fraktion', 'möchte', 'nun', '

In [16]:
try:
    assert isinstance(speech_analysis, dict)
except:
    print("speech_analysis sollte ein `dict` sein.")
try:
    assert len(speech_analysis) == 76
except:
    print("speech_analysis sollte 76 Einträge beinhalten")
try:
    for key in speech_analysis.keys():
        assert isinstance(speech_analysis[key], dict)
        assert 'tokens' in speech_analysis[key].keys()
    print("Alles korrekt")
except:
    print("Jeder Eintrag sollte ein `dict` sein und den Schlüssel `tokens` enthalten")


Alles korrekt


Zuvor haben wir herausgefunden, dass es in der ersten Rede um die Besetzung des Hauptausschusses zu gehen scheint. Mithilfe des Dictionaries `speech_analysis` können wir nun herausfinden, ob der Hauptausschuss auch ein bestimmendes Thema der übrigen Reden war. Dafür bestimmen wir die relative Dokumenthäufigkeit, in dem wir die Anzahl der Dokumente / der Reden, in denen dieser Begriff auftritt, durch die Gesamtanzahl der Reden teilen. Die nachfolgende Zelle zeigt dies am Beispiel:

In [22]:
# die Gesamtzahl der Reden entspricht der Länge des Dictionaries speech_analysis
count_all_speeches = len(speech_analysis)
# Initialisierung einer Variable, die die Anzahl der Reden mit dem Term 'Hauptausschuss' enthält
count_speeches_term = 0

# Iteration über alle Reden
for _, speech_data in speech_analysis.items():
    # Prüfung, ob der Term 'Hauptausschuss' in der Liste der Tokens enthalten ist
    if 'Hauptausschuss' in speech_data['tokens']:
        # Falls ja, wird die Variable count_speeches_term um 1 erhöht
        count_speeches_term += 1

# Berechnung der relativen Dokumenthäufigkeit
rel_doc_freq = count_speeches_term / count_all_speeches
print(rel_doc_freq)


0.07894736842105263


## Ausblick

Die nächste Übungseinheit baut auf den Inhalten dieser Einheit auf. In Ergänzung dazu sollen die statistischen Berechnungen für alle Plenarprotokolle durchgeführt werden – zudem wird die Python-Bibliothek [NLTK](https://www.nltk.org) (Natural Language Toolkit) vorgestellt, die bspw. das Entfernen häufig auftretender Tokens wie 'der' oder 'die' mithilfe sog. Stoppwortlisten erlaubt.